# Convert fore names to simplified (one word) fore names

In [1]:
import collections
import string
import pandas

import pubmedpy.names

In [2]:
name_df_paths = [
    'data/pubmed/authors.tsv.xz',
    'data/pmc/authors.tsv.xz',
    'data/iscb/keynotes.tsv',
]
path_to_df = {}
for path in name_df_paths:
    name_df = pandas.read_csv(path, keep_default_na=False, na_values=[''], sep='\t')
    path_to_df[path] = name_df

In [3]:
def simplify_names(dfs, name_column):
    names = pandas.concat(df[name_column] for df in dfs)
    simple_name_df = names.value_counts(dropna=True).reset_index()
    simple_name_df.columns = [name_column, 'n_authors']
    assert simple_name_df[name_column].isna().sum() == 0
    simplifier = getattr(pubmedpy.names, f'simplify_{name_column}')
    simple_name_df[f'{name_column}_simple'] = simple_name_df[name_column].map(lambda x: simplifier(x, lower=True))
    simple_name_df = simple_name_df.sort_values(name_column)
    return simple_name_df

In [4]:
simple_fore_name_df = simplify_names(path_to_df.values(), 'fore_name')
simple_fore_name_df

,fore_name,n_authors,fore_name_simple
30705,(Max) Zong-Ming,1,max
33242,", Yuxin",1,yuxin
24,A,373,None
2167,A A,17,None
23284,A Aldo,2,aldo
...,...,...,...
22361,Špela,2,špela
14827,Žiga,2,žiga
28940,Živa,1,živa
18167,Živadin,2,živadin


In [5]:
simple_last_name_df = simplify_names(path_to_df.values(), 'last_name')
simple_last_name_df

,last_name,n_authors,last_name_simple
36213,Rozowsky,1,rozowsky
34877,Slater,1,slater
34546,'t Hart,1,t hart
2418,'t Hoen,14,t hoen
37799,(Holly) Yang,1,holly) yang
...,...,...,...
25685,Žárský,2,žárský
36502,železný,1,železný
36257,žurauskienė,1,žurauskienė
22299,‘t Hoen,2,‘t hoen


In [6]:
simple_fore_name_df.sample(n=20, random_state=0).sort_values('n_authors', ascending=False)

,fore_name,n_authors,fore_name_simple
454,Giulia,64,giulia
1534,Akinori,23,akinori
2725,Shinya,14,shinya
7204,Mayya,6,mayya
10963,Yongbing,4,yongbing
10719,Zelmina,4,zelmina
10609,Hajer,4,hajer
10821,Cheng-Hsun,4,cheng-hsun
12404,Anu G.,3,anu
14478,Keyuan,2,keyuan


In [7]:
simple_last_name_df.sample(n=20, random_state=0).sort_values('n_authors', ascending=False)

,last_name,n_authors,last_name_simple
102,Cohen,127,cohen
1396,Heckerman,21,heckerman
1552,Woo,19,woo
4291,Marín,9,marín
9316,Bähler,5,bähler
11076,Hemenway,4,hemenway
11207,Renou,4,renou
12910,Vervier,4,vervier
11362,Slawski,4,slawski
9867,Sakamoto,4,sakamoto


In [8]:
# Most common last names
(
    simple_last_name_df
    .groupby('last_name_simple')
    .n_authors.sum()
    .sort_values(ascending=False)
    .reset_index()
    .head(3)
)

,last_name_simple,n_authors
0,wang,3232
1,li,2716
2,zhang,2652


In [9]:
simple_fore_name_df.to_csv('data/names/fore-names.tsv.xz', sep='\t', index=False)
simple_last_name_df.to_csv('data/names/last-names.tsv.xz', sep='\t', index=False)